In [1]:
import pandas as pd
import numpy as np
import joblib
from keras.models import load_model

from adversarialdefence.mutation_testing import AdversarialDetectorThroughMutation
from adversarialdefence.model_mut_operators import ModelMutationOperators

from adversarialdefence.utils import ModelUtils, GeneralUtils

### ***Data Loading***

In [2]:
df = pd.read_csv('../csv/CICIDS2017_improved-preprocessed.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.shape

(1715326, 49)

In [3]:
df_benign = df[df['Label'] == 0]
df_anomalous = df[df['Label'] == 1]

print(f'Number of benign samples: {df_benign.shape[0]}')
print(f'Number of anomalous samples: {df_anomalous.shape[0]}')

Number of benign samples: 1432918
Number of anomalous samples: 282408


### ***Load models and scalers***

In [4]:
autoencoder = load_model('../modelli/autoencoder_best_weights_96-96.hdf5')
dnn_retrained = load_model('../modelli/dnn_retrained_best_weights.hdf5')
dnn = load_model('../modelli/DNN_best_weights_99.hdf5')

In [5]:
std_scaler_aut = joblib.load('../modelli/std_scaler_aut.bin')
std_scaler_dnn = joblib.load('../modelli/std_scaler_dnn.bin')
columns = df.copy().drop('Label', axis=1).columns

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### ***Define detectors***

In [6]:
MUTATED_MODELS_BASE_PATH = '../modelli/mutation/'
adv_detector_dnn = AdversarialDetectorThroughMutation(dnn, 'DNN', 'SUPERVISED', 0.5, MUTATED_MODELS_BASE_PATH)
adv_detector_aut = AdversarialDetectorThroughMutation(autoencoder, 'AUT', 'UNSUPERVISED', 0.05, MUTATED_MODELS_BASE_PATH)

In [7]:
X_benign = df_benign.sample(n=1000)
y_benign = X_benign.pop('Label')

X_benign_dnn = std_scaler_dnn.transform(X_benign)
X_benign_aut = std_scaler_aut.transform(X_benign)

In [8]:
LCR_th_dnn = adv_detector_dnn.fit(X_benign_dnn, 500, 0.3)
LCR_th_aut = adv_detector_dnn.fit(X_benign_aut, 500, 0.006)

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### ***Load BB Adversarial samples***

In [8]:
BASE_ADVS_CSV_PATH = '../csv/'

In [9]:
df_hsj, X_hsj, y_hsj = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'hsj_new.csv')
df_zoo, X_zoo, y_zoo = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'zoo_new.csv')
df_boundary, X_boundary, y_boundary = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'boundary_new.csv')
df_query_eff, X_query_eff, y_query_eff = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'query_eff_new.csv')

In [10]:
preds_hsj_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_hsj)
preds_zoo_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_zoo)
preds_boundary_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_boundary)
preds_query_eff_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_query_eff)

preds_hsj_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_hsj, 0.035)
preds_zoo_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_zoo, 0.035)
preds_boundary_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_boundary, 0.035)
preds_query_eff_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_query_eff, 0.035)

In [11]:
advs_hsj_dnn_retrained = GeneralUtils.get_advs_samples(preds_hsj_dnn_retrained, df_hsj)
advs_zoo_dnn_retrained = GeneralUtils.get_advs_samples(preds_zoo_dnn_retrained, df_zoo)
advs_boundary_dnn_retrained = GeneralUtils.get_advs_samples(preds_boundary_dnn_retrained, df_boundary)
advs_query_eff_dnn_retrained = GeneralUtils.get_advs_samples(preds_query_eff_dnn_retrained, df_query_eff)

advs_hsj_aut_retrained = GeneralUtils.get_advs_samples(preds_hsj_aut_retrained, df_hsj)
advs_zoo_aut_retrained = GeneralUtils.get_advs_samples(preds_zoo_aut_retrained, df_zoo)
advs_boundary_aut_retrained = GeneralUtils.get_advs_samples(preds_boundary_aut_retrained, df_boundary)
advs_query_eff_aut_retrained = GeneralUtils.get_advs_samples(preds_query_eff_aut_retrained, df_query_eff)

In [9]:
X_benign = df_benign.sample(n=10000)
y_benign = X_benign.pop('Label')

X_anomalous = df_anomalous.sample(n=10000)
y_anomalous = X_anomalous.pop('Label')

X_benign_dnn = std_scaler_dnn.transform(X_benign)
X_benign_aut = std_scaler_aut.transform(X_benign)

X_anomalous_dnn = std_scaler_dnn.transform(X_anomalous)
X_anomalous_aut = std_scaler_aut.transform(X_anomalous)

In [12]:
bb_attacks_dnn_retrained_dict = {
    'HOPSKIPJUMP': advs_hsj_dnn_retrained, 
    'BOUNDARY': advs_boundary_dnn_retrained, 
    'ZOO': advs_zoo_dnn_retrained, 
    'QUERY_EFF': advs_query_eff_dnn_retrained
}

bb_attacks_aut_retrained_dict = {
    'HOPSKIPJUMP': advs_hsj_aut_retrained, 
    'BOUNDARY': advs_boundary_aut_retrained, 
    'ZOO': advs_zoo_aut_retrained, 
    'QUERY_EFF': advs_query_eff_aut_retrained
}

### ***Performance of Target System (Adversarial Training + Mutation Testing)***

In [10]:
def getCommonAdversarialSamples(X_advs, df_advs):
    preds_advs_dnn = ModelUtils.binary_preds_supervised(dnn_retrained, X_advs)
    advs_samples_dnn = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn), df_advs)

    preds_advs_dnn_by_aut = ModelUtils.binary_preds_unsupervised(autoencoder, advs_samples_dnn.copy().drop('Label', axis=1), 0.035)
    return GeneralUtils.get_advs_samples(np.array(preds_advs_dnn_by_aut), advs_samples_dnn)

def getFirstLayerFalseBenignSamples(preds_dnn, preds_aut):
    X_anomalous_with_preds = X_anomalous.assign(preds_dnn=preds_dnn.astype(bool), preds_aut=np.array(preds_aut).astype(bool))
    X_false_benign = X_anomalous_with_preds[(X_anomalous_with_preds['preds_dnn'] == False) & (X_anomalous_with_preds['preds_aut'] == False)]   

    X_false_benign = X_false_benign.drop(['preds_dnn', 'preds_aut'], axis=1)
    return X_false_benign

def getFirstLayerRealBenignSamples(preds_dnn, preds_aut):
    X_benign_with_preds = X_benign.assign(preds_dnn=preds_dnn.astype(bool), preds_aut=np.array(preds_aut).astype(bool))
    X_real_benign = X_benign_with_preds[(X_benign_with_preds['preds_dnn'] == False) & (X_benign_with_preds['preds_aut'] == False)]   

    X_real_benign = X_real_benign.drop(['preds_dnn', 'preds_aut'], axis=1)
    return X_real_benign

def perform_detection(X, attack_type, n_step, do_scaling=False):
    num_detected_list = []
    for i in range(0, n_step):
        if do_scaling:
            detect_status_dnn = adv_detector_dnn.detect(std_scaler_dnn.transform(X), 200, 0.009)
            #detect_status_aut = adv_detector_aut.detect(std_scaler_aut.transform(X), 200, 0.7)
        else:
            detect_status_dnn = adv_detector_dnn.detect(X, 200, 0.009)
            #detect_status_aut = adv_detector_aut.detect(X, 200, 0.7)

        #df_detect_status = pd.DataFrame({
        #    'dnn': detect_status_dnn,
        #    'aut': detect_status_aut
        #}).reset_index(drop=True)

        num_detected_list.append(len([d for d in detect_status_dnn if d == True]))
        #num_detected_list.append(df_detect_status[(df_detect_status['dnn'] == True) | (df_detect_status['aut'] == True)].shape[0])
    
    avg_detected = round(sum(num_detected_list) / len(num_detected_list))
    print(f'Average number of aversarial samples of type {attack_type} detected in {n_step} steps: {avg_detected} / {X.shape[0]} - Ratio {(avg_detected/X.shape[0])*100}%') 

In [14]:
df_advs_hsj_common = getCommonAdversarialSamples(X_hsj, df_hsj)
df_advs_boundary_common = getCommonAdversarialSamples(X_boundary, df_boundary)
df_advs_zoo_common = getCommonAdversarialSamples(X_zoo, df_zoo)
df_advs_query_eff_common = getCommonAdversarialSamples(X_query_eff, df_query_eff)

In [21]:
perform_detection(df_advs_hsj_common.copy().drop('Label',axis=1), 'HOPSKIPJUMP', 1)
perform_detection(df_advs_boundary_common.copy().drop('Label',axis=1), 'BOUNDARY', 1)
perform_detection(df_advs_zoo_common.copy().drop('Label',axis=1), 'ZOO', 1)
perform_detection(df_advs_query_eff_common.copy().drop('Label',axis=1), 'QUERY EFF', 1)

Average number of aversarial samples of type HOPSKIPJUMP detected in 1 steps: 2 / 7 - Ratio 28.57142857142857%
Average number of aversarial samples of type BOUNDARY detected in 1 steps: 2 / 9 - Ratio 22.22222222222222%
Average number of aversarial samples of type ZOO detected in 1 steps: 3 / 10 - Ratio 30.0%
Average number of aversarial samples of type QUERY EFF detected in 1 steps: 0 / 7 - Ratio 0.0%


In [22]:
preds_anomalous_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_anomalous_dnn)
preds_anomalous_aut_retrained = ModelUtils.binary_preds_supervised(autoencoder, X_anomalous_aut, 0.035)

X_first_layer_false_benign = getFirstLayerFalseBenignSamples(preds_anomalous_dnn_retrained, preds_anomalous_aut_retrained)
perform_detection(X_first_layer_false_benign, 'FALSE BENIGN', 1, True)

Average number of aversarial samples of type FALSE BENIGN detected in 1 steps: 0 / 17 - Ratio 0.0%


In [23]:
preds_benign_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_benign_dnn)
preds_benign_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_benign_aut, 0.035)

X_first_layer_real_benign = getFirstLayerRealBenignSamples(preds_benign_dnn_retrained, preds_benign_aut_retrained)
perform_detection(X_first_layer_real_benign, 'REAL BENIGN', 1, True)

Average number of aversarial samples of type REAL BENIGN detected in 1 steps: 2037 / 9247 - Ratio 22.02876608629826%


### ***Attempt 2***

In [18]:
X_benign = df_benign.sample(n=1000)
y_benign = X_benign.pop('Label')

X_benign_dnn = std_scaler_dnn.transform(X_benign)
X_benign_aut = std_scaler_aut.transform(X_benign)

LCR_th_dnn = adv_detector_dnn.fit(X_benign_dnn, 500, 0.1)
#LCR_th_aut = adv_detector_dnn.fit(X_benign_aut, 500, 0.006)

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
perform_detection(df_advs_hsj_common.copy().drop('Label',axis=1), 'HOPSKIPJUMP', 1)
perform_detection(df_advs_boundary_common.copy().drop('Label',axis=1), 'BOUNDARY', 1)
perform_detection(df_advs_zoo_common.copy().drop('Label',axis=1), 'ZOO', 1)
perform_detection(df_advs_query_eff_common.copy().drop('Label',axis=1), 'QUERY EFF', 1)

Average number of aversarial samples of type HOPSKIPJUMP detected in 1 steps: 4 / 7 - Ratio 57.14285714285714%
Average number of aversarial samples of type BOUNDARY detected in 1 steps: 2 / 9 - Ratio 22.22222222222222%
Average number of aversarial samples of type ZOO detected in 1 steps: 2 / 10 - Ratio 20.0%
Average number of aversarial samples of type QUERY EFF detected in 1 steps: 0 / 7 - Ratio 0.0%


In [14]:
X_benign = df_benign.sample(n=10000)
y_benign = X_benign.pop('Label')

X_anomalous = df_anomalous.sample(n=100000)
y_anomalous = X_anomalous.pop('Label')

X_benign_dnn = std_scaler_dnn.transform(X_benign)
X_benign_aut = std_scaler_aut.transform(X_benign)

X_anomalous_dnn = std_scaler_dnn.transform(X_anomalous)
X_anomalous_aut = std_scaler_aut.transform(X_anomalous)

In [15]:
preds_anomalous_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_anomalous_dnn)
preds_anomalous_aut_retrained = ModelUtils.binary_preds_supervised(autoencoder, X_anomalous_aut, 0.035)

X_first_layer_false_benign = getFirstLayerFalseBenignSamples(preds_anomalous_dnn_retrained, preds_anomalous_aut_retrained)
perform_detection(X_first_layer_false_benign, 'FALSE BENIGN', 1, True)

Average number of aversarial samples of type FALSE BENIGN detected in 1 steps: 37 / 120 - Ratio 30.833333333333336%


In [36]:
preds_benign_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_benign_dnn)
preds_benign_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_benign_aut, 0.035)

X_first_layer_real_benign = getFirstLayerRealBenignSamples(preds_benign_dnn_retrained, preds_benign_aut_retrained)
perform_detection(X_first_layer_real_benign, 'REAL BENIGN', 1, True)

Average number of aversarial samples of type REAL BENIGN detected in 1 steps: 46 / 9193 - Ratio 0.5003807244642663%
